# Inhibitory plasticity

## Initialization

In [ ]:
!pip install numpy scipy matplotlib ipywidgets scikit-learn --quiet
import numpy as np
import scipy.linalg as lin
from numpy.random import default_rng
rng = default_rng()
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
plt.style.use("https://github.com/comp-neural-circuits/plasticity-workshop/raw/dev/plots_style.txt")

## Utility functions

In [ ]:
def inhSTDP_analytic(eta,rho,tau,Delta_t):
    """    
    Computes the change in weight for a single pre-post pair due to inhibitory STPD
    
    Parameters:
    tau (number) : plasticity timescale
    eta (number) : learning rate 
    rho (number) : target rate
    Delta_t (1D numpy array) : differences between time post spike and time pre spike
    
    Returns 
    DeltaW (1D numpy array) : the weight change due to the inhibitory STDP rule
                              for each time differenence in tau
    """
    # Calculate DeltaW
    alpha = 2*tau*rho
    DeltaW = eta * ( (1/(2*tau)) * np.exp(-np.abs(Delta_t)/tau) - alpha)
    return DeltaW
    
def inhibitory_stdp_interactive_plot(eta=1.0,rho=1.0,tau=20E-3):
    rate_protocol = 0.5
    xmax = 0.5
    dplot = xmax/30
    x = np.arange(-xmax,xmax,dplot)
    y = np.fromiter(map(lambda _x : inhibitory_stdp_pairing(_x,rate_protocol,5.,tau,eta,rho)[2] , x),dtype=np.float64)
    # let's add the analytic! 
    y_an = inhSTDP_analytic(eta,rho,tau,x)
    # analyti refers to one pair only, but numeric depends on rate of presentation
    y_an *= rate_protocol
    
    x *= 1000 # switch to ms
    xmax *= 1000 # switch to ms
    plt.figure()
    plt.plot([-xmax, xmax], [0, 0], 'k', linestyle=':')
    #plt.plot([0, 0], [-80, 100], 'k', linestyle=':')
    plt.plot(x, y, 'r')
    plt.plot(x,y_an, 'y--')
    plt.xlabel(r'$\Delta t=$ t$_{\mathrm{post}}$ - t$_{\mathrm{pre}}$ (ms)',fontsize=16)
    plt.ylabel(r'$\Delta $W ',fontsize=16)
    plt.title('The inhibitory STDP rule', fontsize=18, fontweight='bold')
    plt.show()
    return
    
def inhSTDP_plot(eta, rho, tau):
    """    
    Generates a plot for the inhibitory plasticity rule
    
    Parameters:
    eta : 
    rho : 
    tau : 
    
    Returns:
    A plot for the inhibitory STDP rule
    """
    Delta_t =  np.linspace(-200, 200, 100)
    # Calculate the synaptic change via the specific function
    DeltaW = inhSTDP_analytic(eta, rho, tau,Delta_t)
    
    # Plotting
    plt.figure(figsize=(8,5))
    plt.plot([Delta_t[0], Delta_t[-1]], [0, 0], 'k', linestyle=':')
    plt.plot([0, 0], [-80, 100], 'k', linestyle=':')
    plt.plot(Delta_t, DeltaW, 'r')
    plt.xlabel(r'$\Delta t=$ t$_{\mathrm{post}}$ - t$_{\mathrm{pre}}$ (ms)',fontsize=16)
    plt.ylabel(r'$\Delta $W (%)',fontsize=16)
    plt.title('The inhibitory STDP rule', fontsize=18, fontweight='bold')
    plt.show()

    
def inh_analysis_plot(num_small_intervals,post_spike,save_inh_weights, duration,Exkeep,Inkeep):
    '''
    
    ....
    
    
    '''
    post_rate = np.zeros(num_small_intervals)
    inh_weights = np.zeros(num_small_intervals)
    for i in range(num_small_intervals):
        post_rate[i] = np.sum(post_spike[i*duration:(i+1)*duration])/ duration * 1000
        inh_weights[i] = np.mean(save_inh_weights[i*duration:(i+1)*duration])
    Exkeep0 = Exkeep[0:int(duration/10)]
    Inkeep0 = Inkeep[0:int(duration/10)]
    Exkeep1 = Exkeep[int(duration*(num_small_intervals-1/10)):duration*num_small_intervals]
    Inkeep1 = Inkeep[int(duration*(num_small_intervals-1/10)):duration*num_small_intervals]    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, figsize=(8,12));
    ax1.plot(range(num_small_intervals),post_rate)
    ax1.set_xlabel("time (s)", fontsize=12);
    ax1.set_ylabel("Firing rate [Hz]", fontsize=12);
    ax1.set_title("Postsynaptic firing rate", fontsize=14);
    ax2.plot(range(num_small_intervals),inh_weights)
    ax2.set_xlabel("time (s) ", fontsize=12);
    ax2.set_ylabel("Mean inhibitory weights", fontsize=12);
    ax2.set_title("Inhibitory weights", fontsize=14);
    ax3.plot(range(int(duration/10)), Exkeep0/100,'r',linewidth=3)
    ax3.plot(range(int(duration/10)), Inkeep0/100, 'g')
    ax3.plot(range(int(duration/10)), (Inkeep0 + Exkeep0)/100, 'k')
    ax3.set_xlabel("time (ms) ", fontsize=12);
    ax3.set_ylabel('Currents', fontsize=12);
    ax3.set_title("Currents before plasticity", fontsize=14);
    ax4.plot(range(int(duration/10)), Exkeep1/100, 'r')
    ax4.plot(range(int(duration/10)), Inkeep1/100, 'g')
    ax4.plot(range(int(duration/10)), (Inkeep1 + Exkeep1)/100, 'k')
    ax4.set_xlabel("time (ms) ", fontsize=12);
    ax4.set_ylabel('Currents', fontsize=12);
    ax4.set_title("Currents after plasticity", fontsize=14);

## 6. Inhibitory plasticity

We now consider synaptic plasticity *from* inhibitory neurons *to* one excitatory neurons. Careful here! A stronger (positive) synaptic weight means that our output neuron is inhibited *more*.



The STDP rule used for inhibition is symmetric, positive for short time differences between pre and post spikes, and negative as the difference increase.

$$
\Delta W = \eta \left( \frac{1}{2\tau}\, 
\exp\left(-\frac{\Delta t}{\tau}\right) - \alpha\right) \quad
\text{and} \quad
\alpha = 2\,\rho\,\tau
$$

### Online update with inhibitory STPD

Like we did before, we will express the plasticity rule in terms of spike detectors. Let's consider a p**R**esynaptic detector $r(t)$ and a p**O**st-synaptic detector $o(t)$. Therefore:

\begin{align}
\frac{\mathrm d\,r}{\mathrm d \,t} &= -  \frac{r}{\tau} \quad ; \quad 
\text{if presynaptic neuron fires at time $t^*$ :} \quad r(t^*) \leftarrow r+1 \quad \\
\frac{\mathrm d\,o}{\mathrm d \,t} &= -  \frac{o}{\tau} \quad ; \quad 
\text{if postsynaptic neuron fires at time $t^*$ :} \quad o(t^*) \leftarrow o+1
\end{align}

If there are multiple presynaptic neurons $(1,2,\ldots,N)$ we need to consider a separate trace for each of them. Therefore $(r_1(t),r_2(t),\ldots r_N(t))$ ... but let's think worry about that later.

The online inhibitory STDP update ( [Vogels et al, 2011](http://www.sciencemag.org/cgi/doi/10.1126/science.1211095) , supplement)  is defined as follows:

\begin{align} 
W \leftarrow & W + \eta \; (o(t^*) - \alpha) \quad 
&& \text{for *presynaptic* spike at time $t^*$}\\
W \leftarrow & W + \eta \; r(t^*) \quad 
&& \text{for *postsynaptic* spike at time $t^*$}\\
\end{align}

Where $\tau$ is the time constant associated to the plasticity window, $\eta$ is the learning constant, and the parameter $\alpha$ can be reparametrized as $\alpha = 2\;\tau\;\rho$, where $\rho$ here represents a "target rate".

We will now implement this rule for a given pre and post spiketrain.

#### **Exercise :** implement the rule for fixed spiketrains 

In [ ]:
interact(inhSTDP_plot, eta = fixed(1e-4), rho = fixed(0.1), tau = (5,70,0.1));

### Online update with inhibitory STPD

Like we did before, we will express the plasticity rule in terms of spike detectors. Let's consider a p**R**esynaptic detector $r(t)$ and a p**O**st-synaptic detector $o(t)$. Therefore:

\begin{align}
\frac{\mathrm d\,r}{\mathrm d \,t} &= -  \frac{r}{\tau} \quad ; \quad 
\text{if presynaptic neuron fires at time $t^*$ :} \quad r(t^*) \leftarrow r+1 \quad \\
\frac{\mathrm d\,o}{\mathrm d \,t} &= -  \frac{o}{\tau} \quad ; \quad 
\text{if postsynaptic neuron fires at time $t^*$ :} \quad o(t^*) \leftarrow o+1
\end{align}

If there are multiple presynaptic neurons $(1,2,\ldots,N)$ we need to consider a separate trace for each of them. Therefore $(r_1(t),r_2(t),\ldots r_N(t))$ ... but let's think worry about that later.

The online inhibitory STDP update ( [Vogels et al, 2011](http://www.sciencemag.org/cgi/doi/10.1126/science.1211095) , supplement)  is defined as follows:

\begin{align} 
W \leftarrow & W + \eta \; (o(t^*) - \alpha) \quad 
&& \text{for *presynaptic* spike at time $t^*$}\\
W \leftarrow & W + \eta \; r(t^*) \quad 
&& \text{for *postsynaptic* spike at time $t^*$}\\
\end{align}

Where $\tau$ is the time constant associated to the plasticity window, $\eta$ is the learning constant, and the parameter $\alpha$ can be reparametrized as $\alpha = 2\;\tau\;\rho$, where $\rho$ here represents a "target rate".

We will now implement this rule for a given pre and post spiketrain.

#### **Exercise :** implement the rule for fixed spiketrains 

In [ ]:
def inhibitory_stpd_weight_change(train_post,train_pre,tau,eta,rho,discretization_step=1E-4,wstart=0.0):
    """
    Compute the weight change over time according to the inhibitory STDP, for given spike trains.
    (imagine the weight change in an experiment where neurons are forced to fire)
    
    Parameters:
    train_post (1D numpy array) : binary vector with spike train. Each bin has duration `discretization_step`
    train_pre  (1D numpy array) : same but for presynaptic neuron
    tau (number) : plasticity time constant
    eta (nunber) : plasticity rate
    rho (number) : plasticity target-rate
    discretization_step = 1E-4 : size of single time bin, in seconds
    wstart = 0.0  :  weight at t=0 . Note that here the weight does not influence the spiking dynamics in any way
    
    Return
    times (1D numpy array) : -
    w     (1D numpy array) : w[k] is the pre to post synaptic weight at time times[k]  
    """
    
    # compute total time, and generate vector of timesteps
    Ntimes = len(train_post)
    Tend = Ntimes * discretization_step 
    times = np.arange(0,Tend,discretization_step)
    
    # express parameter alpha as a function of tau and rho
    alpha = 2*tau*rho
    
    # Initialize the synaptic weight
    w = np.empty(Ntimes)
    w[0] = wstart
    # Initialize the pre- and postsynaptic event detectors
    r = np.empty(Ntimes)     # presynaptic event detector
    r[0] = 0.0
    o = np.empty(Ntimes)      # postsynaptic event detector
    o[0] = 0.0
    
    w_last = wstart # value of last weight computed 
    
    for i in range(Ntimes):
        w[i] = w_last
        # >>> EXERCISE 
        # if there is a pre-synaptic spike  ... 
        # 1. update the pre-synaptic detector r[i]
        # 2. updated the weight using the postsynaptic trace o[i]
        
        if train_pre[i]==1:     
            r[i] += 1
            w_last += eta*(o[i]-alpha)
        # <<<
        
        # >>> EXERCISE
        # if there is a post-synaptic spike ...
        # 1. update the post-synaptic detector o[i]
        # 2. updated the weight using the presynaptic trace r[i]
        if train_post[i]==1:
            o[i] += 1
            w_last += eta*r[i] 
         
        # Euler step to update traces
        if i < Ntimes-1 :   # cannot update after last timestep
            # >>> EXERCISE 
            # Apply Euler's method to the detector equations
            r[i+1] = r[i] - discretization_step * r[i]/tau
            o[i+1] = o[i] - discretization_step * o[i]/tau
            # <<<
    return times,w

The function below, already given, generates two spike trains with fixed frequencies and a pre-post time difference, as has been done before. This time, however, we apply the inhibitory STDP rule to it. In essence, this simulates the weight change in an hypothetical experiment with a pre-post spiking paradigm.


In [ ]:
def inhibitory_stdp_pairing(DeltaT,rate_protocol,duration_protocol,tau,eta,rho,discretization_step=1E-4):
    
    # generate spike trains with pre-post training protocol
    times,train1,train2 = firstspike_secondspike_protocol(rate_protocol,duration_protocol,DeltaT,
                                                          discretization_step = discretization_step)
    
    # compute weight change
    timesw, w = inhibitory_stpd_weight_change(train2,train1,tau,eta,rho, 
                                              discretization_step=discretization_step,wstart=0.0)           
                   
    # average change of weight per second                                           
    w_mean_change = w[-1]/times[-1]
    return timesw,w, w_mean_change
                                              
    

In [ ]:
interact_manual(inhibitory_stdp_interactive_plot,eta =fixed(1), rho = (0.1,10,0.1) , tau = (5E-3,100E-3,1E-3));